### Step1: Import libraries

In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

### Step2: Convert text to binary

In [6]:
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

### Step3:Convert Binary to text

In [11]:
def binary_to_text(binary_data):
    chars = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if len(char) == 8)

### Step4:Encode message into image using LSB

In [14]:
def encode_image(image_path, secret_message, output_path):
    img = Image.open(image_path).convert("RGB")
    data = np.array(img)

    binary_secret = text_to_binary(secret_message) + '1111111111111110'  # EOF marker
    flat_data = data.flatten().astype(np.uint16)

    if len(binary_secret) > len(flat_data):
        raise ValueError("Message too long for this image!")

    for i in range(len(binary_secret)):
        flat_data[i] = (flat_data[i] & 0xFFFE) | int(binary_secret[i])

    new_data = flat_data.astype(np.uint8).reshape(data.shape)
    Image.fromarray(new_data).save(output_path)
    print(f"✅ Message successfully encoded in: {output_path}")

    # Show Encoded Image
    plt.imshow(Image.open(output_path))
    plt.axis('off')
    plt.title("🖼️ Encoded Image (subtle changes)")
    plt.show()

### Step5:Decode hidden message from stego image

In [17]:
def decode_image(stego_image_path):
    img = Image.open(stego_image_path).convert("RGB")
    data = np.array(img).flatten()

    binary_data = ''.join(str(value & 1) for value in data)
    eof = binary_data.find('1111111111111110')

    if eof != -1:
        binary_data = binary_data[:eof]
    else:
        print("⚠️ End-of-message marker not found. Output may be corrupted.")

    return binary_to_text(binary_data)

### Step6: Show Difference Image

In [22]:
def show_difference(original_path, stego_path):
    original = np.array(Image.open(original_path))
    stego = np.array(Image.open(stego_path))
    diff = np.abs(original.astype(int) - stego.astype(int))

    plt.imshow(diff)
    plt.axis('off')
    plt.title("🔍 Difference Image (changes are subtle)")
    plt.show()